## 1. Importar Librerías

Importamos:
- `Executor`: Clase base para executors
- `handler`: Decorador para métodos que procesan mensajes
- `WorkflowBuilder`: Constructor de workflows
- `WorkflowContext`: Contexto de ejecución

In [ ]:
import asyncio
import random
from agent_framework import Executor, WorkflowBuilder, WorkflowContext, WorkflowOutputEvent, handler
from typing_extensions import Never

## 2. Dispatcher Executor

### Responsabilidades:
- Validar que el input es una lista válida de enteros
- Distribuir la lista a múltiples executors en paralelo

### Clase `Executor`:
- Hereda de `Executor` base
- Usa el decorador `@handler` para métodos procesadores
- Recibe `WorkflowContext` para enviar mensajes

In [ ]:
class Dispatcher(Executor):
    """
    The sole purpose of this executor is to dispatch the input of the workflow to
    other executors.
    """

    @handler
    async def handle(self, numbers: list[int], ctx: WorkflowContext[list[int]]):
        if not numbers:
            raise RuntimeError("Input must be a valid list of integers.")

        await ctx.send_message(numbers)

print("✅ Dispatcher definido")

## 3. Average Executor

### Responsabilidades:
- Calcular el promedio de la lista de números
- Enviar el resultado como `float`

### Cálculo:
```python
average = sum(numbers) / len(numbers)
```

In [ ]:
class Average(Executor):
    """Calculate the average of a list of integers."""

    @handler
    async def handle(self, numbers: list[int], ctx: WorkflowContext[float]):
        average: float = sum(numbers) / len(numbers)
        await ctx.send_message(average)

print("✅ Average definido")

## 4. Sum Executor

### Responsabilidades:
- Calcular la suma total de la lista de números
- Enviar el resultado como `int`

### Cálculo:
```python
total = sum(numbers)
```

In [ ]:
class Sum(Executor):
    """Calculate the sum of a list of integers."""

    @handler
    async def handle(self, numbers: list[int], ctx: WorkflowContext[int]):
        total: int = sum(numbers)
        await ctx.send_message(total)

print("✅ Sum definido")

## 5. Aggregator Executor

### Responsabilidades:
- Recibir resultados de múltiples executors upstream
- Combinar resultados en una lista
- Emitir el output final del workflow

### Fan-In automático:
El framework automáticamente:
- Recolecta mensajes de todos los executors upstream
- Los entrega como una lista
- Espera a que todos completen antes de invocar el handler

### Type annotation importante:
```python
results: list[int | float]
```
- Union type porque recibimos `int` (de Sum) y `float` (de Average)

In [ ]:
class Aggregator(Executor):
    """Aggregate the results from the different tasks and yield the final output."""

    @handler
    async def handle(self, results: list[int | float], ctx: WorkflowContext[Never, list[int | float]]):
        """Receive the results from the source executors.

        The framework will automatically collect messages from the source executors
        and deliver them as a list.

        Args:
            results (list[int | float]): execution results from upstream executors.
                The type annotation must be a list of union types that the upstream
                executors will produce.
            ctx (WorkflowContext[Never, list[int | float]]): A workflow context that can yield the final output.
        """
        await ctx.yield_output(results)

print("✅ Aggregator definido")

## 6. Construir el Workflow Fan-Out/Fan-In

### Métodos especiales del WorkflowBuilder:

#### `add_fan_out_edges(source, [targets])`
- Distribuye la salida del `source` a múltiples `targets` en paralelo
- Todos los targets reciben el mismo mensaje
- Se ejecutan concurrentemente

#### `add_fan_in_edges([sources], target)`
- Recolecta salidas de múltiples `sources`
- Espera a que todos completen
- Entrega todos los resultados al `target` como lista

### Arquitectura del workflow:
```
Input → Dispatcher ─┬─→ Average ─┬─→ Aggregator → Output
                    └─→ Sum     ─┘
```

In [ ]:
async def build_and_run_workflow():
    """Construye y ejecuta el workflow concurrente"""
    
    # 1) Create the executors
    dispatcher = Dispatcher(id="dispatcher")
    average = Average(id="average")
    summation = Sum(id="summation")
    aggregator = Aggregator(id="aggregator")

    # 2) Build a simple fan out and fan in workflow
    workflow = (
        WorkflowBuilder()
        .set_start_executor(dispatcher)
        .add_fan_out_edges(dispatcher, [average, summation])
        .add_fan_in_edges([average, summation], aggregator)
        .build()
    )

    print("✅ Workflow construido")
    print("\nEstructura:")
    print("  Input → Dispatcher")
    print("            ├─→ Average ─┐")
    print("            └─→ Sum ─────┴─→ Aggregator → Output")
    
    return workflow

print("✅ Función build_and_run_workflow() definida")

## 7. Ejecutar el Workflow

### Input:
- Lista de 10 números aleatorios entre 1 y 100

### Procesamiento:
1. Dispatcher distribuye la lista
2. Average y Sum procesan en **paralelo**
3. Aggregator combina resultados

### Output esperado:
- Lista con dos elementos: `[promedio, suma]`

In [ ]:
async def main() -> None:
    """Función principal que ejecuta el workflow"""
    
    # Generar números aleatorios
    numbers = [random.randint(1, 100) for _ in range(10)]
    
    print("\n" + "="*70)
    print("WORKFLOW CONCURRENTE - FAN-OUT/FAN-IN")
    print("="*70)
    print(f"\nInput: {numbers}")
    print(f"Total de números: {len(numbers)}")
    
    # Construir workflow
    workflow = await build_and_run_workflow()
    
    # 3) Run the workflow
    print("\n🔄 Ejecutando workflow...\n")
    output: list[int | float] | None = None
    async for event in workflow.run_stream(numbers):
        if isinstance(event, WorkflowOutputEvent):
            output = event.data

    # Mostrar resultados
    if output is not None:
        print("\n" + "="*70)
        print("RESULTADOS")
        print("="*70)
        print(f"\nOutput del workflow: {output}")
        print(f"\n📊 Análisis:")
        print(f"   Promedio calculado: {output[0]:.2f}")
        print(f"   Suma calculada: {output[1]}")
        
        # Verificación manual
        expected_avg = sum(numbers) / len(numbers)
        expected_sum = sum(numbers)
        print(f"\n✅ Verificación:")
        print(f"   Promedio correcto: {abs(output[0] - expected_avg) < 0.01}")
        print(f"   Suma correcta: {output[1] == expected_sum}")

print("✅ Función main() definida")
print("\n⚠️ Ejecuta la siguiente celda para correr el workflow")

In [ ]:
# Ejecutar el workflow
await main()

## 8. Ejecutar Múltiples Veces

Vamos a ejecutar el workflow varias veces con diferentes datos para ver la consistencia:

In [ ]:
async def run_multiple_tests(n_tests: int = 3):
    """Ejecuta el workflow múltiples veces"""
    
    # Construir workflow una sola vez
    workflow = await build_and_run_workflow()
    
    print(f"\n{'='*70}")
    print(f"EJECUTANDO {n_tests} PRUEBAS")
    print(f"{'='*70}\n")
    
    for i in range(1, n_tests + 1):
        numbers = [random.randint(1, 100) for _ in range(10)]
        
        print(f"\nPrueba {i}:")
        print(f"  Input: {numbers}")
        
        # Ejecutar workflow
        async for event in workflow.run_stream(numbers):
            if isinstance(event, WorkflowOutputEvent):
                output = event.data
                print(f"  Promedio: {output[0]:.2f}")
                print(f"  Suma: {output[1]}")

await run_multiple_tests(3)

## Conclusión

Este ejemplo demuestra:

### 1. **Patrón Fan-Out/Fan-In**
- **Fan-Out**: Distribuir trabajo a múltiples executors en paralelo
- **Fan-In**: Recolectar y agregar resultados de múltiples fuentes
- Procesamiento concurrente para mejor performance

### 2. **Executors con Clases**
- Herencia de `Executor` base
- Decorador `@handler` para métodos procesadores
- Identificadores únicos con parámetro `id`

### 3. **Type Safety**
- Union types: `list[int | float]`
- Type hints en WorkflowContext
- Validación en tiempo de compilación

### 4. **Agregación Automática**
- Framework recolecta mensajes automáticamente
- Espera a que todos los upstream completen
- Entrega resultados como lista ordenada

### Aplicaciones Prácticas:

✅ **Análisis de datos**: Calcular múltiples estadísticas en paralelo

✅ **Procesamiento de imágenes**: Aplicar múltiples filtros simultáneamente

✅ **APIs paralelas**: Llamar múltiples servicios y agregar respuestas

✅ **Machine Learning**: Entrenar múltiples modelos en paralelo

✅ **ETL**: Extraer de múltiples fuentes y consolidar

### Ventajas del patrón:

| Aspecto | Secuencial | Fan-Out/Fan-In |
|---------|-----------|----------------|
| Velocidad | 🐌 Lento | ⚡ Rápido |
| Uso de CPU | 📊 Bajo | 📊📊📊 Alto |
| Complejidad | 😊 Simple | 🤔 Moderada |
| Escalabilidad | ❌ Limitada | ✅ Excelente |

### Consideraciones:

⚠️ **Sincronización**: Todos los paths deben completar

⚠️ **Errores**: Si un path falla, todo el workflow puede fallar

⚠️ **Recursos**: Más executors = más memoria

✅ **Performance**: Ideal para operaciones I/O bound

### Extensiones posibles:

🔹 Agregar más executors en paralelo

🔹 Implementar timeouts para cada path

🔹 Manejar fallos parciales

🔹 Agregar logging y métricas por executor

🔹 Combinar con agentes LLM para análisis inteligente